In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_groq import ChatGroq

import os
from dotenv import load_dotenv
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()

True

In [6]:
system = """

Você é um assistente muito útil que irá me ajudar a estudar para o ENEM.

"""

"""

Modelos disponíveis:

- mixtral-8x7b-32768
- llama-guard-3-8b -> Somente para guardrails
- llama-3.2-90b-text-preview
- llama3-70b-8192
- llama-3.1-70b-versatile

"""


llm = ChatGroq(model="llama-3.2-90b-text-preview", temperature=0)


system_prompt = """
Você vai receber uma descrição de vaga de emprego e a cultura dessa empresa.
"""

messages = [
    ("system", system_prompt),
    ("system", "Aqui está descrição da vaga : {descricao}"),
    ("system", "Aqui está o texto relacionado a cultura da empresa: {descricao_cultura}"),
    ("system", "Você irá criar uma biografia que comtemple as duas descrições em apenas um texto."),
    ("system", "Coloque no texto os princpais pontos para fazer o match com o perfil do candidato"),
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | llm | StrOutputParser()

In [7]:
descricao = """
Vaga: Desenvolvedor(a) de Software Júnior
Descrição da Vaga: Estamos em busca de um(a) Desenvolvedor(a) de Software Júnior para integrar nosso time de tecnologia! Se você é apaixonado(a) por programação, gosta de resolver problemas complexos e busca constantemente aprimorar suas habilidades, essa oportunidade é para você. Na nossa empresa, você terá a chance de colaborar com um time dinâmico e inovador, participando do desenvolvimento e da manutenção de sistemas escaláveis e de alto desempenho.

Responsabilidades:

Desenvolver e manter aplicações, trabalhando no back-end e/ou front-end conforme o projeto.
Colaborar com designers e outros desenvolvedores para aprimorar funcionalidades e garantir uma excelente experiência do usuário.
Escrever e revisar código limpo, eficiente e bem documentado.
Participar de sessões de brainstorming para soluções criativas e inovadoras.
Trabalhar em um ambiente ágil e colaborativo, seguindo metodologias como Scrum e Kanban.
Requisitos:

Conhecimento em linguagens de programação como JavaScript, Python, ou Java.
Experiência com frameworks modernos de desenvolvimento web (React, Angular, Vue.js) ou back-end (Django, Node.js).
Habilidades básicas em SQL e bancos de dados relacionais.
Boa comunicação e disposição para trabalhar em equipe.
Formação completa ou em andamento em Ciência da Computação, Engenharia de Software, ou áreas relacionadas.
Desejável:

Experiência com versionamento de código (Git).
Noções de metodologias ágeis.
Interesse por novas tecnologias e inovação.
Oferecemos:

Ambiente flexível e aberto a novas ideias.
Programa de mentoria e desenvolvimento contínuo.
Benefícios como vale-refeição, vale-transporte, plano de saúde e incentivo à educação.
Horário de trabalho flexível e possibilidade de home office.
"""


descricao_cultura = """
Cultura da Empresa:
Na nossa empresa, acreditamos que inovação e colaboração são essenciais para criar produtos impactantes. 
Valorizamos a diversidade, a inclusão e o desenvolvimento contínuo de todos os nossos colaboradores. 
Nosso ambiente é construído sobre pilares de respeito, transparência e autonomia, e incentivamos nossa equipe a compartilhar ideias, 
explorar novas abordagens e desenvolver soluções criativas. Além disso, estamos comprometidos em criar um impacto positivo tanto 
internamente quanto para a sociedade, investindo em ações sustentáveis e no desenvolvimento de projetos que beneficiem a 
comunidade ao nosso redor. Aqui, cada pessoa tem espaço para crescer, contribuir e transformar o futuro da tecnologia!
"""

In [9]:
result = chain.invoke({"descricao" : descricao, "descricao_cultura": descricao_cultura})


In [18]:
# chain create embeddings

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS



embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
embedding_size = 3072

In [22]:
from uuid import uuid4
from langchain_core.documents import Document

# Criação de documentos
document_1 = Document(
    page_content="Engenheiro de software experiente com mais de 5 anos em desenvolvimento full-stack, especializado em Python, JavaScript e React.",
    metadata={"source": "currículo"},
)

document_2 = Document(
    page_content="Gerente de projetos com histórico comprovado na coordenação de equipes multifuncionais e entrega de projetos no prazo e dentro do orçamento.",
    metadata={"source": "currículo"},
)

document_3 = Document(
    page_content="Cientista de dados especializado em aprendizado de máquina, análise estatística e visualização de dados. Proficiente em Python, R e SQL.",
    metadata={"source": "currículo"},
)

document_4 = Document(
    page_content="Especialista em marketing focado em estratégias digitais, campanhas de mídia social e criação de conteúdo. Experiente em SEO e publicidade PPC.",
    metadata={"source": "currículo"},
)

document_5 = Document(
    page_content="Designer gráfico com expertise no Adobe Creative Suite e sólida experiência em branding, narrativa visual e design UX/UI.",
    metadata={"source": "currículo"},
)

document_6 = Document(
    page_content="Profissional de recursos humanos com experiência em recrutamento, relações com colaboradores e gestão de talentos. Conhecido por fortes habilidades interpessoais.",
    metadata={"source": "currículo"},
)

document_7 = Document(
    page_content="Analista financeiro com experiência em previsão, orçamento e modelagem financeira. Proficiente em Excel, Power BI e SAP.",
    metadata={"source": "currículo"},
)

document_8 = Document(
    page_content="Desenvolvedor front-end com expertise em HTML, CSS, JavaScript e paixão por criar websites responsivos e acessíveis.",
    metadata={"source": "currículo"},
)

document_9 = Document(
    page_content="Analista de negócios experiente, com habilidades em melhoria de processos, análise de dados e gestão de stakeholders. Conhecido pela atenção aos detalhes.",
    metadata={"source": "currículo"},
)

document_10 = Document(
    page_content="Representante de atendimento ao cliente com mais de 3 anos de experiência em gerenciamento de solicitações, resolução de reclamações e prestação de serviço excepcional.",
    metadata={"source": "currículo"},
)


# Lista de documentos
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

# Geração de IDs únicos
uuids = [str(uuid4()) for _ in range(len(documents))]

db = FAISS.from_documents(documents, embeddings)

In [23]:
query = result
docs_retriver = db.similarity_search(query, k=5)

In [24]:
docs_retriver

[Document(metadata={'source': 'currículo'}, page_content='Engenheiro de software experiente com mais de 5 anos em desenvolvimento full-stack, especializado em Python, JavaScript e React.'),
 Document(metadata={'source': 'currículo'}, page_content='Desenvolvedor front-end com expertise em HTML, CSS, JavaScript e paixão por criar websites responsivos e acessíveis.'),
 Document(metadata={'source': 'currículo'}, page_content='Cientista de dados especializado em aprendizado de máquina, análise estatística e visualização de dados. Proficiente em Python, R e SQL.'),
 Document(metadata={'source': 'currículo'}, page_content='Profissional de recursos humanos com experiência em recrutamento, relações com colaboradores e gestão de talentos. Conhecido por fortes habilidades interpessoais.'),
 Document(metadata={'source': 'currículo'}, page_content='Analista de negócios experiente, com habilidades em melhoria de processos, análise de dados e gestão de stakeholders. Conhecido pela atenção aos detalhe

In [ ]:
# 

In [67]:
curriculo_pdf = "../data/curriculo_HUDSON.pdf"
curriculo_docs = "../data/curriculo_HUDSON.docx"


In [ ]:
from llama_parse import LlamaParse

import joblib
import os
import nest_asyncio 
nest_asyncio.apply()

parsingInstructionsInterviewer = """
O documento a seguir é um currículo de um possível candidato em busca de vagas.
No documento, haverá uma seção contendo as Habilidades do candidato e outra contendo as Atividades Extracurriculares dele.
Extraia o que conseguir dessas partes
"""

In [63]:
llamaparse = LlamaParse(parsing_instruction=parsingInstructionsInterviewer, result_type="markdown")


In [64]:
parsed_result = llamaparse.get_json_result(curriculo_pdf)

Started parsing the file under job_id 95611df1-35a6-46b5-bebe-231f4c9686a7


In [ ]:
    
# all_informations
all_text = ""
for i in range(len(parsed_result[0]['pages'])):
    all_text += parsed_result[0]['pages'][i]['md']

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Activity(BaseModel):
    """
    Uma descrição detalhada de uma atividade e o papel do candidato nela.
    """
    descricao: str = Field(description="Descrição da atividade")
    papel: str = Field(description="Papel desempenhado pelo candidato na atividade")

class Topics(BaseModel):
    """
    Listas de possíveis tópicos importantes para um candidato.
    """
    habilidades : List[Activity]
    

class Question(BaseModel):
    questoes : str = Field(description="Enunciado da questão")
    
class Questions(BaseModel):
    """
    Questões relacionadas a descrição e papel do candidato e lista de altertivas
    """
    questoes : List[Question]


In [203]:
from langchain_core.prompts import ChatPromptTemplate

messages = [
    (
        "system",
        "Você é um assistente útil que ajuda a identificar as habilidades, atividades de um candidato com base no seguinte texto: {curriculo}.",
    ),
    (
        "system",
        "Aqui estão os tópicos:"
    )
]

prompt = ChatPromptTemplate.from_messages(messages)

chat_model = llm.with_structured_output(Topics)

chain_structured = prompt | chat_model
response = chain_structured.invoke({"curriculo": all_text})


In [204]:
habilidades_json = [{"descricao": res.descricao,"papel":res.papel} for res in response.habilidades]
questions_model = llm.with_structured_output(Questions)

messages = [
    ("system", "Você é um assistem de RH, seu papel é avaliar candidatos"),
    (
        "system",
        """Você vai receber uma lista que possui descrição e papel de um candidato, 
        crie questões para o candidado responder e mostrar que possui essas habilidades esses conhecimentos.
        Exemplo de questão: 
        Desenvolvimento Full-Stack:

        "Descreva um projeto em que você utilizou Python e JavaScript juntos. Como você integrou essas linguagens e quais desafios enfrentou?"
        "Quais são as melhores práticas que você adota para garantir a segurança e eficiência de uma aplicação full-stack?"
        Gerenciamento de Projetos:

        "Conte sobre uma situação em que precisou ajustar o escopo de um projeto para manter o prazo e o orçamento. Como foi o processo de tomada de decisão?"
        "Como você garante a comunicação eficaz entre as equipes multifuncionais durante o andamento de um projeto?"
        Ciência de Dados:

        "Explique como você selecionaria as variáveis mais importantes para um modelo preditivo. Quais técnicas você usaria?"
        "Descreva uma situação em que você precisou usar análise estatística para resolver um problema complexo. Quais métodos você escolheu e por quê?"
        Marketing Digital:

        "Quais métricas você considera mais importantes ao avaliar o desempenho de uma campanha de mídia social? Como você ajustaria a campanha com base nesses dados?"
        "Como você desenvolve estratégias de SEO para atrair público qualificado?"
        Design Gráfico e UX/UI:

        "Fale sobre um projeto onde foi necessário criar uma identidade visual desde o início. Como você lidou com as expectativas do cliente e as necessidades dos usuários?"
        "Qual é o seu processo para garantir que o design de uma interface seja intuitivo e acessível para todos os usuários?"
        Recursos Humanos:

        "Descreva como você aborda um processo de recrutamento para garantir que os candidatos sejam culturalmente alinhados com a empresa."
        "Como você gerencia conflitos entre colaboradores, garantindo um ambiente de trabalho saudável?"
        Desenvolvimento Front-End:

        "Quais são as práticas que você usa para garantir que um site seja responsivo em dispositivos de diferentes tamanhos?"
        "Descreva uma situação em que você precisou otimizar o desempenho de um site. Quais técnicas e ferramentas você utilizou?"
        Análise de Negócios:

        "Como você lida com requisitos ambíguos ao criar melhorias de processo? Qual é sua abordagem para alinhar expectativas com as partes interessadas?"
        "Explique como você utiliza a análise de dados para identificar oportunidades de crescimento ou redução de custos em uma empresa."
        
        Aqui está a lista\n {habilidades}.""",),
    (
        "system",
        "Gere 3 questões no máximo. Aqui estão as questões:\n"
    )
]

prompt = ChatPromptTemplate.from_messages(messages)

chain_structured = prompt | questions_model
response = chain_structured.invoke({"habilidades": habilidades_json})

In [205]:
response    

Questions(questoes=[Question(questoes='Descreva um projeto em que você utilizou Django para desenvolver uma aplicação BackEnd. Quais desafios você enfrentou e como os superou?'), Question(questoes='Explique como você abordaria a classificação de sentimentos em um conjunto de dados. Quais técnicas e algoritmos você utilizaria?'), Question(questoes='Como você integraria uma aplicação com Twitter e Spotify, considerando as restrições de API e segurança?')])